# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [173]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_correlation(): 
    
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]#seleccion de las filas y columnas de cities que interesan
    
    #clean
    nhl_df_clean=nhl_df[(nhl_df['year']==2018)&(~nhl_df['team'].str.contains(r'Division', regex=True, case=False))].iloc[:,[0,2,3]].reset_index(drop=True)
                #filtra para que year==2018 y team NO (~) contenga la cadena 'Division' #muestra todas las filas pero sólo columnas 0,2,3 #resetea indice
    nhl_df_clean['team']=nhl_df_clean['team'].apply(lambda x: re.sub(r'\*', '', x).split()[-1])#elimina el asterisco, divide palabras y se queda con la ultima palabra
    nhl_df_clean['W/L Ratio']=nhl_df_clean['W'].astype(float)/(nhl_df_clean['W'].astype(float)+nhl_df_clean['L'].astype(float))#creacion de ratio
    nhl_df_clean.rename(columns={"team": "NHL"}, inplace=True)#cambiar nombre para campo comun con mismo nombre
    
    cities_clean=cities[(cities['NHL']!=chr(8212))&(~cities['NHL'].str.contains(r'^\[note[^\]]*\]', regex=True, case=False))].iloc[:,[0,1,5]].reset_index(drop=True)
    cities_clean['NHL']=cities_clean['NHL'].apply(lambda x: re.sub(r'\[[^\]]*\]', '', x).split()[-1])
    cities_clean.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True)#cambiar nombre para mayor legibilidad
    
    #añadir manualmente los equipos que faltan a las areas metropolitanas
    nuevas_filas = pd.DataFrame({'Metropolitan area': ['New York City', 'New York City', 'Los Angeles'],
                                 'Population': ['20153634', '20153634', '13310447'],
                                 'NHL':['Rangers','Islanders','Kings']
                                })
    #diccionario_poblacion = cities_clean.set_index('Metropolitan area')['Population'].to_dict()#crear diccionario de poblaciones
    cities_clean = pd.concat([cities_clean, nuevas_filas], ignore_index=True)
    cities_clean['Population'] = pd.to_numeric(cities_clean['Population'], errors='coerce')
    #merge dfs
    df_merge = pd.merge(cities_clean, nhl_df_clean, on='NHL', how='right')
    #una fila por area con Groupby
    df_group=df_merge.groupby('Metropolitan area')#para poder acceder a las columnas de un Groupby hay que transformarlo en df. 
    resultado_final=df_group.agg({'Population':np.mean,'NHL': np.size, 'W/L Ratio': np.mean}).sort_values(by='Population', ascending=False).reset_index() #de este modo resultado_final es un df y con reset_index sacamos 
                                                                                                                                                            #la variable de agrupacion del indice
    #print(resultado_final)   
    #raise NotImplementedError()

    #--------------------------------------------------------------------------------------------------------------
    population_by_region = list(resultado_final['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(resultado_final['W/L Ratio']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]#hay que extraer el coeficiente solo sin pval
nhl_correlation()

C:\Users\User\AppData\Local\Temp\ipykernel_10628\1789210980.py:35: FutureWarning: The provided callable <function mean at 0x0000028B6609BA60> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  resultado_final=df_group.agg({'Population':np.mean,'NHL': np.size, 'W/L Ratio': np.mean}).sort_values(by='Population', ascending=False).reset_index() #de este modo resultado_final es un df y con reset_index sacamos
C:\Users\User\AppData\Local\Temp\ipykernel_10628\1789210980.py:35: FutureWarning: The provided callable <function mean at 0x0000028B6609BA60> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  resultado_final=df_group.agg({'Population':np.mean,'NHL': np.size, 'W/L Ratio': np.mean}).sort_values(by='Population', ascending=False).reset_index() #de

0.012486162921209914

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [197]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nba_correlation():

    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nba_df_clean=nba_df[(nba_df['year']==2018)].iloc[:,[0,1,2,3]].reset_index(drop=True)
    nba_df_clean['team']=nba_df_clean['team'].apply(lambda x: re.sub(r'\([^\]]*\)', '', x))
    nba_df_clean['team']=nba_df_clean['team'].apply(lambda x: re.sub(r'\*', '', x).split()[-1])
    nba_df_clean['W/L Ratio']=nba_df_clean['W'].astype(float)/(nba_df_clean['W'].astype(float)+nba_df_clean['L'].astype(float))
    nba_df_clean.rename(columns={"team": "NBA"}, inplace=True)

    cities_clean=cities[(cities['NBA']!=chr(8212))&(~cities['NBA'].str.contains(r'^\[note[^\]]*\]', regex=True, case=False))].iloc[:,[0,1,4]].reset_index(drop=True)
    cities_clean['NBA']=cities_clean['NBA'].apply(lambda x: re.sub(r'\[[^\]]*\]', '', x).split()[-1])
    cities_clean.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True)#cambiar nombre para mayor legibilidad

    nuevas_filas = pd.DataFrame({'Metropolitan area': ['New York City','Los Angeles'],
                                 'Population': ['20153634','13310447'],
                                 'NBA':['Knicks','Lakers']
                                })
    #diccionario_poblacion = cities_clean.set_index('Metropolitan area')['Population'].to_dict()#crear diccionario de poblaciones
    cities_clean = pd.concat([cities_clean, nuevas_filas], ignore_index=True)
    cities_clean['Population'] = pd.to_numeric(cities_clean['Population'], errors='coerce')

    #merge dfs
    df_merge = pd.merge(cities_clean, nba_df_clean, on='NBA', how='right')
    #una fila por area con Groupby
    df_group=df_merge.groupby('Metropolitan area')#para poder acceder a las columnas de un Groupby hay que transformarlo en df. 
    resultado_final=df_group.agg({'Population':np.mean,'NBA': np.size, 'W/L Ratio': np.mean}).sort_values(by='Population', ascending=False).reset_index()
    
    #raise NotImplementedError()
    
    population_by_region = list(resultado_final['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(resultado_final['W/L Ratio']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nba_correlation()

C:\Users\User\AppData\Local\Temp\ipykernel_10628\632197834.py:35: FutureWarning: The provided callable <function mean at 0x0000028B6609BA60> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  resultado_final=df_group.agg({'Population':np.mean,'NBA': np.size, 'W/L Ratio': np.mean}).sort_values(by='Population', ascending=False).reset_index()
C:\Users\User\AppData\Local\Temp\ipykernel_10628\632197834.py:35: FutureWarning: The provided callable <function mean at 0x0000028B6609BA60> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  resultado_final=df_group.agg({'Population':np.mean,'NBA': np.size, 'W/L Ratio': np.mean}).sort_values(by='Population', ascending=False).reset_index()


PearsonRResult(statistic=-0.17657160252844617, pvalue=0.3687474160446289)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [226]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def mlb_correlation(): 
    
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    mlb_df_clean=mlb_df[(mlb_df['year']==2018)].iloc[:,[0,1,2,3]].reset_index(drop=True)
    mlb_df_clean['team']=mlb_df_clean['team'].apply(lambda x: x.split()[-1] if 'Sox' not in x else (x.split()[-2]+' '+x.split()[-1]) )
    mlb_df_clean['W/L Ratio']=mlb_df_clean['W'].astype(float)/(mlb_df_clean['W'].astype(float)+mlb_df_clean['L'].astype(float))
    mlb_df_clean.rename(columns={"team": "MLB"}, inplace=True)

    cities_clean=cities[(cities['MLB']!=chr(8212))&(~cities['MLB'].str.contains(r'^\[note[^\]]*\]', regex=True, case=False))].iloc[:,[0,1,3]].reset_index(drop=True)
    cities_clean['MLB']=cities_clean['MLB'].apply(lambda x: re.sub(r'\[[^\]]*\]', '', x).split()[-1] if 'Sox' not in x else (re.sub(r'\[[^\]]*\]', '', x).split()[-2]+' '+re.sub(r'\[[^\]]*\]', '', x).split()[-1]))
    cities_clean.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True)#cambiar nombre para mayor legibilidad

    nuevas_filas = pd.DataFrame({'Metropolitan area': ['Chicago','Los Angeles','San Francisco Bay Area','New York City'],
                                 'Population': ['9512999','13310447','6657982','20153634'],
                                 'MLB':['Cubs','Dodgers','Giants','Yankees']
                                })
    
    cities_clean = pd.concat([cities_clean, nuevas_filas], ignore_index=True)
    cities_clean['Population'] = pd.to_numeric(cities_clean['Population'], errors='coerce')

    #merge dfs
    df_merge = pd.merge(cities_clean, mlb_df_clean, on='MLB', how='right')
    #una fila por area con Groupby
    df_group=df_merge.groupby('Metropolitan area')
    resultado_final=df_group.agg({'Population':np.mean,'MLB': np.size, 'W/L Ratio': np.mean}).sort_values(by='Population', ascending=False).reset_index()
    
    #raise NotImplementedError()
    
    population_by_region = list(resultado_final['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(resultado_final['W/L Ratio'])

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
mlb_correlation()

C:\Users\User\AppData\Local\Temp\ipykernel_10628\1819660018.py:34: FutureWarning: The provided callable <function mean at 0x0000028B6609BA60> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  resultado_final=df_group.agg({'Population':np.mean,'MLB': np.size, 'W/L Ratio': np.mean}).sort_values(by='Population', ascending=False).reset_index()
C:\Users\User\AppData\Local\Temp\ipykernel_10628\1819660018.py:34: FutureWarning: The provided callable <function mean at 0x0000028B6609BA60> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  resultado_final=df_group.agg({'Population':np.mean,'MLB': np.size, 'W/L Ratio': np.mean}).sort_values(by='Population', ascending=False).reset_index()


0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [260]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nfl_correlation(): 

    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nfl_df_clean=nfl_df[(nfl_df['year']==2018)&(~nfl_df['team'].str.contains(r'FC', regex=True, case=False))].iloc[:,[13,11,1]].reset_index(drop=True)
    nfl_df_clean['team']=nfl_df_clean['team'].apply(lambda x: re.sub(r'[\*\+]', '', x).split()[-1])
    nfl_df_clean['W/L Ratio']=nfl_df_clean['W'].astype(float)/(nfl_df_clean['W'].astype(float)+nfl_df_clean['L'].astype(float))
    nfl_df_clean.rename(columns={"team": "NFL"}, inplace=True)

    cities_clean=cities[(cities['NFL']!=chr(8212))&(~cities['NFL'].str.contains(r'^\[note[^\]]*\]', regex=True, case=False))].iloc[:,[0,1,2]].drop(13).reset_index(drop=True)
    cities_clean['NFL']=cities_clean['NFL'].apply(lambda x: re.sub(r'\[[^\]]*\]', '', x).split()[-1])
    cities_clean.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True)

    nuevas_filas = pd.DataFrame({'Metropolitan area': ['New York City','Los Angeles','San Francisco Bay Area'],
                                 'Population': ['20153634','13310447','6657982'],
                                 'NFL':['Giants','Rams','49ers']
                                })

    cities_clean = pd.concat([cities_clean, nuevas_filas], ignore_index=True)
    cities_clean['Population'] = pd.to_numeric(cities_clean['Population'], errors='coerce')

    #merge dfs
    df_merge = pd.merge(cities_clean, nfl_df_clean, on='NFL', how='right')
    #una fila por area con Groupby
    df_group=df_merge.groupby('Metropolitan area')
    resultado_final=df_group.agg({'Population':np.mean,'NFL': np.size, 'W/L Ratio': np.mean}).sort_values(by='Population', ascending=False).reset_index()
    
    #raise NotImplementedError()
    
    population_by_region = list(resultado_final['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(resultado_final['W/L Ratio'])

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nfl_correlation()

C:\Users\User\AppData\Local\Temp\ipykernel_10628\69929925.py:33: FutureWarning: The provided callable <function mean at 0x0000028B6609BA60> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  resultado_final=df_group.agg({'Population':np.mean,'NFL': np.size, 'W/L Ratio': np.mean}).sort_values(by='Population', ascending=False).reset_index()
C:\Users\User\AppData\Local\Temp\ipykernel_10628\69929925.py:33: FutureWarning: The provided callable <function mean at 0x0000028B6609BA60> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  resultado_final=df_group.agg({'Population':np.mean,'NFL': np.size, 'W/L Ratio': np.mean}).sort_values(by='Population', ascending=False).reset_index()


0.004922112149349414

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [12]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def sports_team_performance():

    mlb_df=pd.read_csv("assets/mlb.csv")
    nhl_df=pd.read_csv("assets/nhl.csv")
    nba_df=pd.read_csv("assets/nba.csv")
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nhl_df_clean=nhl_df[(nhl_df['year']==2018)&(~nhl_df['team'].str.contains(r'Division', regex=True, case=False))].iloc[:,[0,2,3]].reset_index(drop=True)
    nhl_df_clean['team']=nhl_df_clean['team'].apply(lambda x: re.sub(r'\*', '', x).split()[-1])
    nhl_df_clean['W/L Ratio']=nhl_df_clean['W'].astype(float)/(nhl_df_clean['W'].astype(float)+nhl_df_clean['L'].astype(float))
    nhl_df_clean.rename(columns={"team": "NHL"}, inplace=True)
    
    cities_clean=cities[(cities['NHL']!=chr(8212))&(~cities['NHL'].str.contains(r'^\[note[^\]]*\]', regex=True, case=False))].iloc[:,[0,1,5]].reset_index(drop=True)
    cities_clean['NHL']=cities_clean['NHL'].apply(lambda x: re.sub(r'\[[^\]]*\]', '', x).split()[-1])
    cities_clean.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True)
    
    nuevas_filas = pd.DataFrame({'Metropolitan area': ['New York City', 'New York City', 'Los Angeles'],
                                 'Population': ['20153634', '20153634', '13310447'],
                                 'NHL':['Rangers','Islanders','Kings']
                                })
    cities_clean = pd.concat([cities_clean, nuevas_filas], ignore_index=True)
    cities_clean['Population'] = pd.to_numeric(cities_clean['Population'], errors='coerce')
    #merge dfs
    df_merge = pd.merge(cities_clean, nhl_df_clean, on='NHL', how='right')
    df_group=df_merge.groupby('Metropolitan area')
    NHL=df_group.agg({'Population':np.mean,'NHL': np.size, 'W/L Ratio': np.mean}).sort_values(by='Population', ascending=False).reset_index()
    #------------------------------------------------------------------------------------------------------------------------------------------------------
    nba_df_clean=nba_df[(nba_df['year']==2018)].iloc[:,[0,1,2,3]].reset_index(drop=True)
    nba_df_clean['team']=nba_df_clean['team'].apply(lambda x: re.sub(r'\([^\]]*\)', '', x))
    nba_df_clean['team']=nba_df_clean['team'].apply(lambda x: re.sub(r'\*', '', x).split()[-1])
    nba_df_clean['W/L Ratio']=nba_df_clean['W'].astype(float)/(nba_df_clean['W'].astype(float)+nba_df_clean['L'].astype(float))
    nba_df_clean.rename(columns={"team": "NBA"}, inplace=True)

    cities_clean=cities[(cities['NBA']!=chr(8212))&(~cities['NBA'].str.contains(r'^\[note[^\]]*\]', regex=True, case=False))].iloc[:,[0,1,4]].reset_index(drop=True)
    cities_clean['NBA']=cities_clean['NBA'].apply(lambda x: re.sub(r'\[[^\]]*\]', '', x).split()[-1])
    cities_clean.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True)#cambiar nombre para mayor legibilidad

    nuevas_filas = pd.DataFrame({'Metropolitan area': ['New York City','Los Angeles'],
                                 'Population': ['20153634','13310447'],
                                 'NBA':['Knicks','Lakers']
                                })
    cities_clean = pd.concat([cities_clean, nuevas_filas], ignore_index=True)
    cities_clean['Population'] = pd.to_numeric(cities_clean['Population'], errors='coerce')
    #merge dfs
    df_merge = pd.merge(cities_clean, nba_df_clean, on='NBA', how='right')
    #una fila por area con Groupby
    df_group=df_merge.groupby('Metropolitan area')
    NBA=df_group.agg({'Population':np.mean,'NBA': np.size, 'W/L Ratio': np.mean}).sort_values(by='Population', ascending=False).reset_index()
    #-----------------------------------------------------------------------------------------------------------------------------------------------
    mlb_df_clean=mlb_df[(mlb_df['year']==2018)].iloc[:,[0,1,2,3]].reset_index(drop=True)
    mlb_df_clean['team']=mlb_df_clean['team'].apply(lambda x: x.split()[-1] if 'Sox' not in x else (x.split()[-2]+' '+x.split()[-1]) )
    mlb_df_clean['W/L Ratio']=mlb_df_clean['W'].astype(float)/(mlb_df_clean['W'].astype(float)+mlb_df_clean['L'].astype(float))
    mlb_df_clean.rename(columns={"team": "MLB"}, inplace=True)

    cities_clean=cities[(cities['MLB']!=chr(8212))&(~cities['MLB'].str.contains(r'^\[note[^\]]*\]', regex=True, case=False))].iloc[:,[0,1,3]].reset_index(drop=True)
    cities_clean['MLB']=cities_clean['MLB'].apply(lambda x: re.sub(r'\[[^\]]*\]', '', x).split()[-1] if 'Sox' not in x else (re.sub(r'\[[^\]]*\]', '', x).split()[-2]+' '+re.sub(r'\[[^\]]*\]', '', x).split()[-1]))
    cities_clean.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True)#cambiar nombre para mayor legibilidad

    nuevas_filas = pd.DataFrame({'Metropolitan area': ['Chicago','Los Angeles','San Francisco Bay Area','New York City'],
                                 'Population': ['9512999','13310447','6657982','20153634'],
                                 'MLB':['Cubs','Dodgers','Giants','Yankees']
                                })
    cities_clean = pd.concat([cities_clean, nuevas_filas], ignore_index=True)
    cities_clean['Population'] = pd.to_numeric(cities_clean['Population'], errors='coerce')
    #merge dfs
    df_merge = pd.merge(cities_clean, mlb_df_clean, on='MLB', how='right')
    #una fila por area con Groupby
    df_group=df_merge.groupby('Metropolitan area')
    MLB=df_group.agg({'Population':np.mean,'MLB': np.size, 'W/L Ratio': np.mean}).sort_values(by='Population', ascending=False).reset_index()
    #-------------------------------------------------------------------------------------------------------------------------------------------
    nfl_df_clean=nfl_df[(nfl_df['year']==2018)&(~nfl_df['team'].str.contains(r'FC', regex=True, case=False))].iloc[:,[13,11,1]].reset_index(drop=True)
    nfl_df_clean['team']=nfl_df_clean['team'].apply(lambda x: re.sub(r'[\*\+]', '', x).split()[-1])
    nfl_df_clean['W/L Ratio']=nfl_df_clean['W'].astype(float)/(nfl_df_clean['W'].astype(float)+nfl_df_clean['L'].astype(float))
    nfl_df_clean.rename(columns={"team": "NFL"}, inplace=True)

    cities_clean=cities[(cities['NFL']!=chr(8212))&(~cities['NFL'].str.contains(r'^\[note[^\]]*\]', regex=True, case=False))].iloc[:,[0,1,2]].drop(13).reset_index(drop=True)
    cities_clean['NFL']=cities_clean['NFL'].apply(lambda x: re.sub(r'\[[^\]]*\]', '', x).split()[-1])
    cities_clean.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True)

    nuevas_filas = pd.DataFrame({'Metropolitan area': ['New York City','Los Angeles','San Francisco Bay Area'],
                                 'Population': ['20153634','13310447','6657982'],
                                 'NFL':['Giants','Rams','49ers']
                                })
    cities_clean = pd.concat([cities_clean, nuevas_filas], ignore_index=True)
    cities_clean['Population'] = pd.to_numeric(cities_clean['Population'], errors='coerce')
    #merge dfs
    df_merge = pd.merge(cities_clean, nfl_df_clean, on='NFL', how='right')
    #una fila por area con Groupby
    df_group=df_merge.groupby('Metropolitan area')
    NFL=df_group.agg({'Population':np.mean,'NFL': np.size, 'W/L Ratio': np.mean}).sort_values(by='Population', ascending=False).reset_index()
    #----------------------------------------------------------------------------------------------------------------------------------------------
    NHL['W/L Ratio'] = pd.to_numeric(NHL['W/L Ratio'], errors='coerce')
    NBA['W/L Ratio'] = pd.to_numeric(NBA['W/L Ratio'], errors='coerce')
    MLB['W/L Ratio'] = pd.to_numeric(MLB['W/L Ratio'], errors='coerce')
    NFL['W/L Ratio'] = pd.to_numeric(NFL['W/L Ratio'], errors='coerce')
    
    dataframes = [NHL, NBA, MLB, NFL]
    combinaciones = []
    # Realiza combinaciones utilizando bucles anidados
    for i in range(len(dataframes)):
        for j in range(i+1, len(dataframes)):
            # Realiza una operación de inner join entre los DataFrames i y j
            combinacion = pd.merge(dataframes[i], dataframes[j], on='Metropolitan area', how='inner')
            combinaciones.append(combinacion)
            
    combis=['NHL_NBA','NHL_MLB','NHL_NFL','NBA_MLB','NBA_NFL','MLB_NFL']
    pvals=[]
    for i in range(len(combinaciones)):
        result_name=combis[i]
        result_name=stats.ttest_rel(combinaciones[i]['W/L Ratio_x'],combinaciones[i]['W/L Ratio_y'])
        pvals.append(result_name[1])
    
    p_values = pd.DataFrame()
    p_values['NHL'] = [np.nan,pvals[0],pvals[1],pvals[2]]  
    p_values['NBA'] = [pvals[0],np.nan,pvals[4],pvals[3]] 
    p_values['NFL'] = [pvals[1],pvals[4],np.nan,pvals[5]] 
    p_values['MLB'] = [pvals[2],pvals[3],pvals[5],np.nan]  
    p_values = p_values.set_index(pd.Index(['NHL', 'NBA', 'NFL', 'MLB']))

    #raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    #sports = ['NFL', 'NBA', 'NHL', 'MLB']
    #p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
sports_team_performance()

C:\Users\User\AppData\Local\Temp\ipykernel_2956\4048671786.py:33: FutureWarning: The provided callable <function mean at 0x000001EFDF78C180> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  NHL=df_group.agg({'Population':np.mean,'NHL': np.size, 'W/L Ratio': np.mean}).sort_values(by='Population', ascending=False).reset_index()
C:\Users\User\AppData\Local\Temp\ipykernel_2956\4048671786.py:33: FutureWarning: The provided callable <function mean at 0x000001EFDF78C180> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  NHL=df_group.agg({'Population':np.mean,'NHL': np.size, 'W/L Ratio': np.mean}).sort_values(by='Population', ascending=False).reset_index()
C:\Users\User\AppData\Local\Temp\ipykernel_2956\4048671786.py:55: FutureWarning: The provided 

,NHL,NBA,NFL,MLB
NHL,NaN,0.022297,0.000708,0.030883
NBA,0.022297,NaN,0.941792,0.950540
NFL,0.000708,0.941792,NaN,0.802069
MLB,0.030883,0.950540,0.802069,NaN
